In [23]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 2_PSCC_Policy_Pool_200.xlsx to 2_PSCC_Policy_Pool_200 (1).xlsx
User uploaded file "2_PSCC_Policy_Pool_200 (1).xlsx" with length 22860 bytes


In [25]:
import os
for fn in uploaded.keys():
    print(f"文件 {fn} 存储在: {os.path.join(os.getcwd(), fn)}")

文件 2_PSCC_Policy_Pool_200 (1).xlsx 存储在: /content/2_PSCC_Policy_Pool_200 (1).xlsx


In [27]:
import pandas as pd
from io import BytesIO

# 从上传的文件中读取
excel_file = pd.ExcelFile(BytesIO(uploaded['2_PSCC_Policy_Pool_200 (1).xlsx']))
df = excel_file.parse('2216')  # 这里假设你要读取的工作表是'2216'

In [12]:
!pip install pytesseract
!pip install pdf2image
!pip install python-docx
!pip install PyMuPDF


In [28]:
import pandas as pd
import requests
import io
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from docx import Document
import fitz
import os

# 配置 pytesseract 路径，根据你的安装路径修改
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

# 读取 Excel 文件
excel_file = pd.ExcelFile('/content/2_PSCC_Policy_Pool_200 (1).xlsx')

# 获取指定工作表中的数据
df = excel_file.parse('2216')

# 定义函数：从 URL 下载文件
def download_file(url):
    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            return io.BytesIO(response.content)
    except requests.RequestException as e:
        print(f"下载文件 {url} 时出现错误: {e}")
    return None

# 定义函数：将 PDF 转换为 TXT
def pdf_to_text(pdf_path):
    text = ""
    doc = fitz.open(stream=pdf_path.read(), filetype="pdf")
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# 定义函数：将 JPG 转换为 TXT
def jpg_to_text(jpg_path):
    image = Image.open(jpg_path)
    return pytesseract.image_to_string(image)

# 定义函数：将 DOC 转换为 TXT
def doc_to_text(doc_path):
    doc = Document(doc_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

# 定义函数：将 TXT 转换为 TXT（直接读取）
def txt_to_text(txt_path):
    return txt_path.read().decode('utf-8', errors='ignore')

# 定义函数：根据文件扩展名选择合适的转换函数
def convert_to_text(url):
    file_extension = url.split('.')[-1].lower()
    file_content = download_file(url)
    if file_content is None:
        return ""
    if file_extension == 'pdf':
        return pdf_to_text(file_content)
    elif file_extension == 'jpg':
        return jpg_to_text(file_content)
    elif file_extension == 'doc':
        return doc_to_text(file_content)
    elif file_extension == 'txt':
        return txt_to_text(file_content)
    return ""

# 创建保存转换后文件的目录
if not os.path.exists('/mnt/converted_texts'):
    os.makedirs('/mnt/converted_texts')

# 遍历 URL 列，进行转换并保存为 TXT 文件
for index, row in df.iterrows():
    url = row['Url']
    text = convert_to_text(url)
    file_name = f"{row['Uid']}.txt"
    file_path = os.path.join('/mnt/converted_texts', file_name)
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(text)

下载文件 https://www.moa.gov.cn/ztzl/xzgnylsn/gd/200909/t20090923_1356488.htm 时出现错误: HTTPSConnectionPool(host='www.moa.gov.cn', port=443): Max retries exceeded with url: /ztzl/xzgnylsn/gd/200909/t20090923_1356488.htm (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))


In [ ]:
import os

folder_path = '/mnt/converted_texts'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    print(f"文件 {file_path} 的内容:")
                    for line in f:
                        print(line.strip())
                    print("-" * 50)  # 打印分隔线区分不同文件
            except FileNotFoundError:
                print(f"文件 {file_path} 不存在")
            except UnicodeDecodeError:
                print(f"文件 {file_path} 解码出错")